In [1]:
import numpy as np
from mxnet import nd
from mxnet import image
import sys

sys.path.append('..')
import utils
from time import time
from mxnet import gluon
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from mxnet import init
import random
import mxnet as mx
import math

classes = ['background', 'p1', 'p2', 'p3', 'p4',
           'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
           'p11', 'p12', 'p13', 'p14', 'p15',
           'p16', 'p17', 'p18', 'p19']
colormap = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
            [128, 0, 128], [0, 64, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0],
            [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128],
            [64, 128, 128], [192, 128, 128], [0, 64, 0], [128, 64, 0],
            [0, 192, 0], [128, 192, 0]]
min_coordinate = [[504, 697], [1063, 606], [948, 888], [286, 902], [1079, 1191], [1053, 1507], [1048, 1661],
                  [993, 1729], [1025, 1706], [415, 1391], [1121, 1355], [1138, 1388], [1245, 1294],
                  [1212, 1477], [1170, 1152], [1114, 1756], [635, 1090], [1081, 1100], [350, 1006]]
expand_size = 20
landmark_index = 20

data_root = '../data'
image_root = data_root + '/CephalometricLandmark/PredictCroppedImage96'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
rgb_mean = nd.array([0.485, 0.456, 0.406])
rgb_std = nd.array([0.229, 0.224, 0.225])
init_learning_rate = 0.02

for all_round_index in range(100):
    #     image_root = data_root + '/CephalometricLandmark/ContrastImage1'
    for _ in range(15, 19):
        for landmark_index in range(0, 19):
            #     for landmark_index in range(0, 19):
            #     for landmark_index in [2,3,5,9,12,15]:
            learning_rate = init_learning_rate
            print()


            def read_images(dataset_num=0):

                if dataset_num == 0:
                    begin_index = 1
                    end_index = 151
                elif dataset_num == 1:
                    begin_index = 151
                    end_index = 301
                else:
                    begin_index = 301
                    end_index = 401

                data, label = [None] * (end_index - begin_index), [None] * (end_index - begin_index)
                index = 0
                for i in range(begin_index, end_index):
                    image_filename = image_root + "/%02d/%03d.bmp" % (landmark_index + 1, i)
                    txt_filename1 = txt_root + '/400_junior' + "/%03d.txt" % i
                    txt_filename2 = txt_root + '/400_predict_average' + "/%03d.txt" % i
                    # #         label_image[index] = nd.zeros_like(data[index])

                    with open(txt_filename1, 'r') as f:
                        txts = f.read().split()
                    with open(txt_filename2, 'r') as f:
                        txts1 = f.read().split()
                    #             x = int((int(txts[landmark_index].split(',')[0]) + int(txts1[landmark_index].split(',')[0]))/2) - min_coordinate[landmark_index][0]
                    #             y = int((int(txts[landmark_index].split(',')[1]) + int(txts1[landmark_index].split(',')[1]))/2) - min_coordinate[landmark_index][1]
                    #         label_image[index][y-expand_size:y+expand_size,x-expand_size:x+expand_size] = colormap[landmark_index+1]
                    x = int(txts[landmark_index].split(',')[0]) - int(txts1[landmark_index].split(',')[0]) + int(96/2)
                    y = int(txts[landmark_index].split(',')[1]) - int(txts1[landmark_index].split(',')[1]) + int(96/2)
                    minx = x - expand_size
                    maxx = x + expand_size
                    if minx < 0:
                        minx = 0
                    if minx >= 96:
                        minx = 95
                    if maxx < 0:
                        maxx = 0
                    if maxx >= 96:
                        maxx = 95

                    miny = y - expand_size
                    maxy = y + expand_size
                    if miny < 0:
                        miny = 0
                    if miny >= 96:
                        miny = 95
                    if maxy < 0:
                        maxy = 0
                    if maxy >= 96:
                        maxy = 95
                    data[index] = image.imread(image_filename)
                    label[index] = nd.zeros((data[index].shape[0], data[index].shape[1]))
                    #             print(label[index].shape)
#                     print(miny,maxy,minx,maxx)
                    if miny != maxy and minx != maxx:
                        label[index][miny:maxy, minx:maxx] = 1
                    #             label[index] = nd.flip(label[index],0)
                    #             print(data[index].shape,label[index].shape)
                    index += 1
                return data, label


            def normalize_image(data):
                #             noise = nd.zeros((640, 640, 1))
                #             if random.random() > 0.2:
                #                 mx.random.seed(np.random.randint(1, 1000))
                #                 noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
                #             return ((data.astype('float32')+ 10*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
                return (data.astype('float32') / 255 - rgb_mean) / rgb_std


            class VOCSegDataset(gluon.data.Dataset):

                def __init__(self, dataset_num, crop_size):
                    self.crop_size = crop_size
                    self.data, self.label = read_images(dataset_num=dataset_num)
                    self.data[:] = [normalize_image(im) for im in self.data]

                def __getitem__(self, idx):
                    data = self.data[idx]
                    label = self.label[idx]

                    #                 aug1 = image.HorizontalFlipAug(1)
                    #                 aug2 = image.BrightnessJitterAug(1)
                    #                 aug3 = image.ContrastJitterAug(1)
                    #                 if random.random() > 0.5:
                    #                     data = aug1(data)
                    #                     label = nd.flip(label,1)
                    #                 data = aug2(data)
                    return data.transpose((2, 0, 1)), label

                def __len__(self):
                    return len(self.data)


            input_shape = (96, 96)
            voc_train = VOCSegDataset(0, input_shape)
            voc_test1 = VOCSegDataset(1, input_shape)

            batch_size = 8
            train_data = gluon.data.DataLoader(
                voc_train, batch_size, shuffle=True, last_batch='discard')
            test_data = gluon.data.DataLoader(
                voc_test1, batch_size, last_batch='discard')

            conv = nn.Conv2D(10, kernel_size=4, padding=1, strides=2)
            conv_trans = nn.Conv2DTranspose(3, kernel_size=4, padding=1, strides=2)

            conv.initialize()
            conv_trans.initialize()

            pretrained_net = models.resnet50_v2(pretrained=True)

            net = nn.HybridSequential()
            for layer in pretrained_net.features[:-2]:
                net.add(layer)

            num_classes = len(classes)

            with net.name_scope():
                net.add(
                    nn.Conv2D(2, kernel_size=1),
                    nn.Conv2DTranspose(2, kernel_size=64, padding=16, strides=32)
                )


            def bilinear_kernel(in_channels, out_channels, kernel_size):
                factor = (kernel_size + 1) // 2
                if kernel_size % 2 == 1:
                    center = factor - 1
                else:
                    center = factor - 0.5
                og = np.ogrid[:kernel_size, :kernel_size]
                filt = (1 - abs(og[0] - center) / factor) * \
                       (1 - abs(og[1] - center) / factor)
                weight = np.zeros(
                    (in_channels, out_channels, kernel_size, kernel_size),
                    dtype='float32')
                weight[range(in_channels), range(out_channels), :, :] = filt
                return nd.array(weight)


            conv_trans = net[-1]
            conv_trans.initialize(init=init.Zero())
            net[-2].initialize(init=init.Xavier())

            x = nd.zeros((batch_size, 3, *input_shape))
            net(x)

            shape = conv_trans.weight.data().shape
            conv_trans.weight.set_data(bilinear_kernel(*shape[0:3]))
            round_num = 10
            for round_index in range(round_num):

                loss = gluon.loss.SoftmaxCrossEntropyLoss(axis=1)

                ctx = utils.try_all_gpus()
                net.collect_params().reset_ctx(ctx)
                trainer = gluon.Trainer(net.collect_params(),
                                        'SGD', {'learning_rate': learning_rate, 'wd': 1e-3})

                utils.train(train_data, test_data, net, loss,
                            trainer, ctx, num_epochs=10)


                #             learning_rate = learning_rate / 2

                def predict(im):
                    data = normalize_image(im)
                    #                 data = (im.astype('float32') / 255 - rgb_mean) / rgb_std
                    data = data.transpose((2, 0, 1)).expand_dims(axis=0)
                    yhat = net(data.as_in_context(ctx[0]))
                    pred = nd.argmax(yhat, axis=1)
                    return pred.reshape((pred.shape[1], pred.shape[2]))


                def label2image(pred):
                    x = pred.astype('int32').asnumpy()
                    cm = nd.array(colormap).astype('uint8')
                    return nd.array(cm[x, :])


                def evaluate_acc(result, label, index):
                    if (len(np.where(result.asnumpy() > 0)[1]) == 0 or len(np.where(label.asnumpy() > 0)[1]) == 0):
                        return False, False, False, False
                    #     result_maxx = np.max(np.where(result.asnumpy()>0)[1])
                    #     result_minx = np.min(np.where(result.asnumpy()>0)[1])
                    #     result_maxy = np.max(np.where(result.asnumpy()>0)[0])
                    #     result_miny = np.min(np.where(result.asnumpy()>0)[0])

                    #     result_centerx = int((result_maxx + result_minx)/2)
                    #     result_centery = int((result_maxy + result_miny)/2)
                    txt_filename1 = txt_root + '/400_junior' + "/%03d.txt" % index
                    txt_filename2 = txt_root + '/400_predict_average' + "/%03d.txt" % index

                    with open(txt_filename1, 'r') as f:
                        txts = f.read().split()
                    with open(txt_filename2, 'r') as f:
                        txts1 = f.read().split()
                    x = int(txts[landmark_index].split(',')[0]) - int(txts1[landmark_index].split(',')[0]) + int(96/2)
                    y = int(txts[landmark_index].split(',')[1]) - int(txts1[landmark_index].split(',')[1]) + int(96/2)

                    result_avgx = np.average(np.where(result.asnumpy() > 0)[1])
                    result_avgy = np.average(np.where(result.asnumpy() > 0)[0])

                    #     label_maxx = np.max(np.where(label.asnumpy()>0)[1])
                    #     label_minx = np.min(np.where(label.asnumpy()>0)[1])
                    #     label_maxy = np.max(np.where(label.asnumpy()>0)[0])
                    #     label_miny = np.min(np.where(label.asnumpy()>0)[0])

                    #     label_centerx = int((label_maxx + label_minx)/2)
                    #     label_centery = int((label_maxy + label_miny)/2)

                    #     lable_avgx = np.average(np.where(label.asnumpy()>0)[1])
                    #     lable_avgy = np.average(np.where(label.asnumpy()>0)[0])

                    #     d1 = pow((result_centerx - label_centerx),2) + pow((result_centery - label_centery),2)
                    d2 = pow((result_avgx - x), 2) + pow((result_avgy - y), 2)
                    #     print("============================")
                    #     print(np.where(result.asnumpy()>0))
                    #     print(result_centerx,label_centerx)
                    #     print(result_centery,label_centery)
                    #     print(d1,d2)
                    return (d2 < 400, d2 < 625, d2 < 900, d2 < 1600)


                test_images, test_labels = read_images(1)
                n = len(test_images)
                acc1 = 0
                acc2 = 0
                acc3 = 0
                acc4 = 0

                # print(test_images[0])
                for i in range(n):
                    x = test_images[i]
                    #                 print(i+1)
                    #     print(x.shape)
                    #     start = time()
                    result = predict(x)
                    f1, f2, f3, f4 = evaluate_acc(result, test_labels[i], i + 151)
                    acc1 += int(f1)
                    acc2 += int(f2)
                    acc3 += int(f3)
                    acc4 += int(f4)

                #             print("expand_size : ", expand_size, " round : ", 10 + 10 * round_index, " acc1 : ", acc1 / n, " acc2 : ",
                #                   acc2 / n)
                print("index : ", landmark_index, " expand size  : ", expand_size, " round : ", 10 + 10 * round_index,
                      " test1 acc : ", "%.3f" % (acc1 / n),
                      "%.3f" % (acc2 / n), "%.3f" % (acc3 / n), "%.3f" % (acc4 / n))

                test_images, test_labels = read_images(2)
                n = len(test_images)
                acc1 = 0
                acc2 = 0
                acc3 = 0
                acc4 = 0

                # print(test_images[0])
                for i in range(n):
                    x = test_images[i]
                    #     print(x.shape)
                    #     start = time()
                    result = predict(x)
                    f1, f2, f3, f4 = evaluate_acc(result, test_labels[i], i + 301)
                    acc1 += int(f1)
                    acc2 += int(f2)
                    acc3 += int(f3)
                    acc4 += int(f4)

                #             print("expand_size : ", expand_size, "  round : ", 10 + 10 * round_index, " acc1 : ", acc1 / n, " acc2 : ",
                #                   acc2 / n)
                print("index : ", landmark_index, " expand size  : ", expand_size, " round : ", 10 + 10 * round_index,
                      " test2 acc : ", "%.3f" % (acc1 / n), "%.3f" % (acc2 / n), "%.3f" % (acc3 / n),
                      "%.3f" % (acc4 / n))



index :  0  expand size  :  20  round :  10  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  10  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  20  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  20  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  30  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  30  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  40  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  40  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  50  test1 acc :  0.947 0.953 0.967 0.973
index :  0  expand size  :  20  round :  50  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  60  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  60  test2 acc :  0.980 0.990 0.990 0.990
index :  0  exp

index :  5  expand size  :  20  round :  10  test1 acc :  0.860 0.953 0.973 0.980
index :  5  expand size  :  20  round :  10  test2 acc :  0.870 0.940 0.960 0.980
index :  5  expand size  :  20  round :  20  test1 acc :  0.860 0.953 0.973 0.980
index :  5  expand size  :  20  round :  20  test2 acc :  0.870 0.940 0.960 0.990
index :  5  expand size  :  20  round :  30  test1 acc :  0.867 0.953 0.973 0.980
index :  5  expand size  :  20  round :  30  test2 acc :  0.860 0.940 0.960 0.990
index :  5  expand size  :  20  round :  40  test1 acc :  0.860 0.953 0.973 0.980
index :  5  expand size  :  20  round :  40  test2 acc :  0.870 0.940 0.960 0.990
index :  5  expand size  :  20  round :  50  test1 acc :  0.867 0.953 0.973 0.980
index :  5  expand size  :  20  round :  50  test2 acc :  0.870 0.950 0.960 0.990
index :  5  expand size  :  20  round :  60  test1 acc :  0.867 0.953 0.973 0.980
index :  5  expand size  :  20  round :  60  test2 acc :  0.870 0.940 0.960 0.990
index :  5  expa

index :  10  expand size  :  20  round :  10  test1 acc :  0.867 0.933 0.960 0.987
index :  10  expand size  :  20  round :  10  test2 acc :  0.890 0.930 0.980 0.990
index :  10  expand size  :  20  round :  20  test1 acc :  0.873 0.933 0.953 0.987
index :  10  expand size  :  20  round :  20  test2 acc :  0.880 0.930 0.980 0.990
index :  10  expand size  :  20  round :  30  test1 acc :  0.873 0.933 0.953 0.987
index :  10  expand size  :  20  round :  30  test2 acc :  0.890 0.930 0.980 0.990
index :  10  expand size  :  20  round :  40  test1 acc :  0.873 0.933 0.953 0.987
index :  10  expand size  :  20  round :  40  test2 acc :  0.890 0.930 0.980 0.990
index :  10  expand size  :  20  round :  50  test1 acc :  0.880 0.933 0.953 0.987
index :  10  expand size  :  20  round :  50  test2 acc :  0.890 0.930 0.980 0.990
index :  10  expand size  :  20  round :  60  test1 acc :  0.873 0.933 0.953 0.987
index :  10  expand size  :  20  round :  60  test2 acc :  0.890 0.930 0.980 0.990
inde

index :  14  expand size  :  20  round :  100  test2 acc :  0.720 0.840 0.920 0.970

index :  15  expand size  :  20  round :  10  test1 acc :  0.687 0.807 0.880 0.940
index :  15  expand size  :  20  round :  10  test2 acc :  0.640 0.790 0.860 0.940
index :  15  expand size  :  20  round :  20  test1 acc :  0.687 0.800 0.880 0.940
index :  15  expand size  :  20  round :  20  test2 acc :  0.640 0.780 0.860 0.940
index :  15  expand size  :  20  round :  30  test1 acc :  0.687 0.807 0.880 0.940
index :  15  expand size  :  20  round :  30  test2 acc :  0.640 0.780 0.860 0.940
index :  15  expand size  :  20  round :  40  test1 acc :  0.693 0.807 0.880 0.940
index :  15  expand size  :  20  round :  40  test2 acc :  0.640 0.770 0.860 0.940
index :  15  expand size  :  20  round :  50  test1 acc :  0.693 0.800 0.887 0.940
index :  15  expand size  :  20  round :  50  test2 acc :  0.640 0.770 0.860 0.940
index :  15  expand size  :  20  round :  60  test1 acc :  0.693 0.800 0.887 0.940
in

index :  0  expand size  :  20  round :  100  test1 acc :  0.940 0.953 0.967 0.973
index :  0  expand size  :  20  round :  100  test2 acc :  0.980 0.990 0.990 0.990

index :  1  expand size  :  20  round :  10  test1 acc :  0.760 0.833 0.893 0.953
index :  1  expand size  :  20  round :  10  test2 acc :  0.840 0.910 0.930 0.930
index :  1  expand size  :  20  round :  20  test1 acc :  0.780 0.840 0.887 0.953
index :  1  expand size  :  20  round :  20  test2 acc :  0.840 0.910 0.930 0.930
index :  1  expand size  :  20  round :  30  test1 acc :  0.780 0.840 0.880 0.953
index :  1  expand size  :  20  round :  30  test2 acc :  0.830 0.900 0.930 0.930
index :  1  expand size  :  20  round :  40  test1 acc :  0.780 0.840 0.880 0.953
index :  1  expand size  :  20  round :  40  test2 acc :  0.840 0.900 0.930 0.930
index :  1  expand size  :  20  round :  50  test1 acc :  0.780 0.840 0.880 0.953
index :  1  expand size  :  20  round :  50  test2 acc :  0.830 0.900 0.930 0.930
index :  1  e

...

index :  3  expand size  :  20  round :  80  test1 acc :  0.620 0.740 0.853 0.913
index :  3  expand size  :  20  round :  80  test2 acc :  0.690 0.830 0.850 0.880
index :  3  expand size  :  20  round :  90  test1 acc :  0.620 0.740 0.847 0.913
index :  3  expand size  :  20  round :  90  test2 acc :  0.690 0.830 0.850 0.880
index :  3  expand size  :  20  round :  100  test1 acc :  0.620 0.747 0.847 0.913
index :  3  expand size  :  20  round :  100  test2 acc :  0.690 0.830 0.850 0.880

index :  4  expand size  :  20  round :  10  test1 acc :  0.600 0.753 0.847 0.947
index :  4  expand size  :  20  round :  10  test2 acc :  0.660 0.830 0.890 0.980
index :  4  expand size  :  20  round :  20  test1 acc :  0.587 0.747 0.853 0.947
index :  4  expand size  :  20  round :  20  test2 acc :  0.650 0.820 0.880 0.970
index :  4  expand size  :  20  round :  30  test1 acc :  0.587 0.747 0.853 0.947
index :  4  expand size  :  20  round :  30  test2 acc :  0.660 0.820 0.890 0.980
index :  4  e

index :  8  expand size  :  20  round :  80  test1 acc :  0.853 0.940 0.960 0.993
index :  8  expand size  :  20  round :  80  test2 acc :  0.950 1.000 1.000 1.000
index :  8  expand size  :  20  round :  90  test1 acc :  0.853 0.940 0.960 0.993
index :  8  expand size  :  20  round :  90  test2 acc :  0.950 1.000 1.000 1.000
index :  8  expand size  :  20  round :  100  test1 acc :  0.853 0.940 0.960 0.993
index :  8  expand size  :  20  round :  100  test2 acc :  0.950 1.000 1.000 1.000

index :  9  expand size  :  20  round :  10  test1 acc :  0.653 0.787 0.847 0.920
index :  9  expand size  :  20  round :  10  test2 acc :  0.610 0.740 0.820 0.910
index :  9  expand size  :  20  round :  20  test1 acc :  0.667 0.787 0.840 0.920
index :  9  expand size  :  20  round :  20  test2 acc :  0.610 0.730 0.820 0.910
index :  9  expand size  :  20  round :  30  test1 acc :  0.680 0.787 0.840 0.920
index :  9  expand size  :  20  round :  30  test2 acc :  0.620 0.730 0.820 0.910
index :  9  e

index :  13  expand size  :  20  round :  70  test2 acc :  0.700 0.880 0.940 1.000
index :  13  expand size  :  20  round :  80  test1 acc :  0.860 0.953 0.987 0.993
index :  13  expand size  :  20  round :  80  test2 acc :  0.700 0.880 0.940 1.000
index :  13  expand size  :  20  round :  90  test1 acc :  0.880 0.953 0.987 0.993
index :  13  expand size  :  20  round :  90  test2 acc :  0.690 0.880 0.940 1.000
index :  13  expand size  :  20  round :  100  test1 acc :  0.873 0.953 0.987 0.993
index :  13  expand size  :  20  round :  100  test2 acc :  0.690 0.880 0.940 1.000

index :  14  expand size  :  20  round :  10  test1 acc :  0.853 0.927 0.953 0.973
index :  14  expand size  :  20  round :  10  test2 acc :  0.710 0.830 0.920 0.970
index :  14  expand size  :  20  round :  20  test1 acc :  0.860 0.920 0.947 0.973
index :  14  expand size  :  20  round :  20  test2 acc :  0.730 0.840 0.920 0.970
index :  14  expand size  :  20  round :  30  test1 acc :  0.847 0.913 0.947 0.973
i

index :  18  expand size  :  20  round :  70  test1 acc :  0.580 0.740 0.853 0.913
index :  18  expand size  :  20  round :  70  test2 acc :  0.660 0.760 0.820 0.900
index :  18  expand size  :  20  round :  80  test1 acc :  0.580 0.733 0.853 0.913
index :  18  expand size  :  20  round :  80  test2 acc :  0.660 0.760 0.820 0.900
index :  18  expand size  :  20  round :  90  test1 acc :  0.580 0.740 0.853 0.913
index :  18  expand size  :  20  round :  90  test2 acc :  0.670 0.760 0.820 0.900
index :  18  expand size  :  20  round :  100  test1 acc :  0.580 0.740 0.853 0.913
index :  18  expand size  :  20  round :  100  test2 acc :  0.670 0.760 0.820 0.910

index :  0  expand size  :  20  round :  10  test1 acc :  0.933 0.940 0.953 0.960
index :  0  expand size  :  20  round :  10  test2 acc :  0.980 0.990 0.990 0.990
index :  0  expand size  :  20  round :  20  test1 acc :  0.927 0.947 0.960 0.967
index :  0  expand size  :  20  round :  20  test2 acc :  0.980 0.990 0.990 0.990
index

KeyboardInterrupt: 